In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook
from tensorflow.keras.optimizers import SGD,RMSprop,Adam
from tensorflow import keras
from tensorflow.keras.layers import Dense,LSTM,Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.utils import normalize

In [3]:
from google.colab import files
uploaded=files.upload()

Saving 2 year dataset.xlsx to 2 year dataset.xlsx


In [10]:
data_frame=pd.read_excel('2 year dataset.xlsx',date_parser=True)

In [11]:
data_frame.head(5)

,Date,apple,burger,chocolate,donuts,egg,fish,garlic,honey,ice cream,...,yam,popcorn,nut,tomato,vegetables,peri sauces,mutton,spaghetti,wings,prac
0,1-Jan-2018,0,1,1,0,1,1,0,1,1,...,0,0,1,0,1,0,1,1,1,1
1,2-Jan-2018,0,1,1,1,1,1,0,1,0,...,1,0,0,0,0,0,0,0,0,1
2,3-Jan-2018,1,1,0,0,1,0,1,0,1,...,0,1,0,0,1,0,1,1,1,1
3,4-Jan-2018,1,0,0,1,1,1,1,0,1,...,1,1,0,0,1,0,1,0,0,1
4,5-Jan-2018,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,1,1,1


In [12]:
data_frame.tail(5)

,Date,apple,burger,chocolate,donuts,egg,fish,garlic,honey,ice cream,...,yam,popcorn,nut,tomato,vegetables,peri sauces,mutton,spaghetti,wings,prac
725,27-Dec-2019,1,1,1,1,0,0,1,0,1,...,1,0,1,1,1,0,1,1,1,1
726,28-Dec-2019,0,0,1,0,1,0,0,1,1,...,0,1,1,1,0,1,1,0,0,1
727,29-Dec-2019,1,1,1,1,0,0,1,0,0,...,1,0,0,1,0,0,1,1,1,1
728,30-Dec-2019,0,1,0,1,0,0,1,0,1,...,1,0,1,0,0,1,0,1,0,1
729,31-Dec-2019,1,0,1,0,1,1,0,1,0,...,0,0,1,1,0,1,1,0,1,1


In [13]:
data_frame.set_index('Date',inplace=True)

In [14]:
data_frame.index=pd.to_datetime(data_frame.index)

In [15]:
data_frame.head()

,apple,burger,chocolate,donuts,egg,fish,garlic,honey,ice cream,jelly,...,yam,popcorn,nut,tomato,vegetables,peri sauces,mutton,spaghetti,wings,prac
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-01,0,1,1,0,1,1,0,1,1,0,...,0,0,1,0,1,0,1,1,1,1
2018-01-02,0,1,1,1,1,1,0,1,0,0,...,1,0,0,0,0,0,0,0,0,1
2018-01-03,1,1,0,0,1,0,1,0,1,1,...,0,1,0,0,1,0,1,1,1,1
2018-01-04,1,0,0,1,1,1,1,0,1,0,...,1,1,0,0,1,0,1,0,0,1
2018-01-05,0,0,0,0,1,0,0,0,0,1,...,1,0,0,0,0,0,0,1,1,1


In [16]:
data_frame.tail()

,apple,burger,chocolate,donuts,egg,fish,garlic,honey,ice cream,jelly,...,yam,popcorn,nut,tomato,vegetables,peri sauces,mutton,spaghetti,wings,prac
Date,,,,,,,,,,,,,,,,,,,,,
2019-12-27,1,1,1,1,0,0,1,0,1,0,...,1,0,1,1,1,0,1,1,1,1
2019-12-28,0,0,1,0,1,0,0,1,1,0,...,0,1,1,1,0,1,1,0,0,1
2019-12-29,1,1,1,1,0,0,1,0,0,1,...,1,0,0,1,0,0,1,1,1,1
2019-12-30,0,1,0,1,0,0,1,0,1,1,...,1,0,1,0,0,1,0,1,0,1
2019-12-31,1,0,1,0,1,1,0,1,0,1,...,0,0,1,1,0,1,1,0,1,1


In [19]:
data_frame.count()

apple          730
burger         730
chocolate      730
donuts         730
egg            730
fish           730
garlic         730
honey          730
ice cream      730
jelly          730
ketchup        730
lemons         730
mango          730
oranges        730
pasta          730
yougurt        730
quicee         730
carrot         730
rice           729
olive oil      730
yam            730
popcorn        730
nut            730
tomato         730
vegetables     730
peri sauces    730
mutton         730
spaghetti      730
wings          730
 prac          730
dtype: int64

In [20]:
data_frame.isnull().sum()

apple          0
burger         0
chocolate      0
donuts         0
egg            0
fish           0
garlic         0
honey          0
ice cream      0
jelly          0
ketchup        0
lemons         0
mango          0
oranges        0
pasta          0
yougurt        0
quicee         0
carrot         0
rice           1
olive oil      0
yam            0
popcorn        0
nut            0
tomato         0
vegetables     0
peri sauces    0
mutton         0
spaghetti      0
wings          0
 prac          0
dtype: int64

In [21]:
data_frame['rice'].fillna(1,inplace=True)

In [23]:
train_size=int(len(data_frame)* 0.75)
test_size=len(data_frame)-train_size
train_data,test_data=data_frame.iloc[0:train_size].values,data_frame.iloc[train_size:len(data_frame)].values
print(train_data.shape,test_data.shape)

(547, 30) (183, 30)


In [131]:
scaler = MinMaxScaler()
train_DATA = scaler.fit_transform(train_data)
test_DATA=scaler.fit_transform(test_data)
print(type(train_DATA),type(test_DATA))

<class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [132]:
np.shape(train_DATA),np.shape(test_DATA)

((547, 30), (183, 30))

In [133]:
# def create(feature,label,time_stamp=1):
#     X=[]
#     Y=[]
#     for i in range(len(feature)-time_stamp):
#         v=feature.iloc[i:(i+time_stamp)].to_numpy()
#         X.append(v)
#         Y.append(label.iloc[i+time_stamp])
#     return np.array(X),np.array(Y)

In [134]:
# x_train,y_train=create(train_data,train_data,30)
# print(np.shape(x_train),np.shape(y_train))

In [135]:
# x_test,y_test=create(test_data,test_data,time_stamp=30)
# print(np.shape(x_test),np.shape(y_test))

In [194]:
train_gent=TimeseriesGenerator(train_DATA,train_DATA,length=3,batch_size=1)

In [195]:
train_gent[0][0].shape

(1, 3, 30)

In [196]:
val_gent=TimeseriesGenerator(test_DATA,test_DATA,length=3,batch_size=1)

In [197]:
val_gent[0][0].shape

(1, 3, 30)

In [198]:
# def model_LSTM():
#     regressior = Sequential()
#     regressior.add(LSTM(units = 120, activation = 'relu', return_sequences = True, input_shape = (x_train.shape[1], 30)))
#     regressior.add(Dropout(0.2))
#     regressior.add(LSTM(units = 80, activation = 'relu'))
#     regressior.add(Dropout(0.2))
#     regressior.add(Dense(units = 30))
#     regressior.compile(loss='mean_squared_error',optimizer='adam')
#     return regressior

In [219]:
def model_LSTM():
    regressior = Sequential()
    regressior.add(LSTM(units = 120, activation = 'relu', return_sequences = True, input_shape = (3, 30)))
    regressior.add(Dropout(0.2))
    regressior.add(LSTM(units = 80, activation = 'relu',return_sequences=True))
    regressior.add(Dropout(0.2))
    
    regressior.add(LSTM(units = 80, activation = 'relu',return_sequences=True))
    regressior.add(Dropout(0.2))
    
    regressior.add(LSTM(units = 60, activation = 'relu'))
    regressior.add(Dropout(0.2))
    regressior.add(Dense(units = 30))
    regressior.compile(loss='mse',optimizer=optimizers.Adam(lr=0.001),metrics=['mae'])
    return regressior

In [220]:
model=model_LSTM()

In [221]:
model.fit_generator(train_gent,epochs=5,verbose=1,shuffle=False,validation_data=(val_gent))

Epoch 1/5
544/544 [==============================] - 28s 51ms/step - loss: 0.2706 - mean_absolute_error: 0.4851 - val_loss: 0.2405 - val_mean_absolute_error: 0.4803
Epoch 2/5
544/544 [==============================] - 15s 28ms/step - loss: 0.2467 - mean_absolute_error: 0.4794 - val_loss: 0.2384 - val_mean_absolute_error: 0.4762
Epoch 3/5
544/544 [==============================] - 15s 27ms/step - loss: 0.2454 - mean_absolute_error: 0.4801 - val_loss: 0.2383 - val_mean_absolute_error: 0.4771
Epoch 4/5
544/544 [==============================] - 16s 29ms/step - loss: 0.2438 - mean_absolute_error: 0.4791 - val_loss: 0.2389 - val_mean_absolute_error: 0.4783
Epoch 5/5
544/544 [==============================] - 16s 29ms/step - loss: 0.2433 - mean_absolute_error: 0.4789 - val_loss: 0.2390 - val_mean_absolute_error: 0.4789


In [0]:
# model.fit(x_train,y_train,epochs=30,batch_size=1,validation_data=(x_test,y_test),shuffle=False)

In [222]:
model.predict_generator(val_gent)

array([[0.5273614 , 0.46902752, 0.51958424, ..., 0.52205807, 0.49998057,
        0.00185722],
       [0.5277998 , 0.47002563, 0.5195326 , ..., 0.52295357, 0.50105405,
        0.00197582],
       [0.5168404 , 0.45910537, 0.50909346, ..., 0.51189774, 0.49039444,
        0.00085902],
       ...,
       [0.55399334, 0.49555105, 0.5460233 , ..., 0.5502992 , 0.5266201 ,
        0.00517016],
       [0.51991934, 0.46208057, 0.51224   , ..., 0.51508796, 0.49317652,
        0.00091108],
       [0.5304264 , 0.47234982, 0.522724  , ..., 0.52567804, 0.5033489 ,
        0.00191472]], dtype=float32)